In [1]:
from simple.ohlc import debounce, midPrice, TTrade, npJoin
from simple.plotly import chartFigure
from simple.funcs import symlog
import numpy as np

In [2]:
T = np.load('data/ETHUSDT.2022-05-08.tick.npz')['ETHUSDT'].view(np.recarray)[:2000]
T[-5:]

rec.array([('2022-05-08T00:01:50.167000', 1995, 2638.5 , 2.000e+00),
           ('2022-05-08T00:01:50.167000', 1996, 2638.5 , 5.275e+03),
           ('2022-05-08T00:01:50.167000', 1997, 2638.56, 5.000e+00),
           ('2022-05-08T00:01:50.167000', 1998, 2638.62, 3.704e+03),
           ('2022-05-08T00:01:50.167000', 1999, 2638.63, 3.000e+00)],
          dtype={'names':['DateTimeA','LocalTimeA','PriceA','VolumeA'], 'formats':['<M8[us]','<i8','<f8','<f8'], 'offsets':[0,8,16,24], 'itemsize':40})

In [3]:
Buy = T[T.VolumeA > 0]
Sell = T[T.VolumeA < 0]

In [4]:
C = debounce(T)
C[-5:]

rec.array([('2022-05-08T00:01:47.203000', 196, 2638.395,       0, 9.0000e+01, 9.000e+01,    0.,  2,  2,  0),
           ('2022-05-08T00:01:47.215000', 197, 2638.445,       0, 2.0000e+00, 2.000e+00,    0.,  1,  1,  0),
           ('2022-05-08T00:01:47.281000', 198, 2638.495, 2886000, 1.3646e+04, 1.277e+04, -876., 44, 33, 11),
           ('2022-05-08T00:01:50.167000', 199, 2638.555,       0, 5.0000e+00, 5.000e+00,    0.,  1,  1,  0),
           ('2022-05-08T00:01:50.167000', 200, 2638.615,       0, 3.7070e+03, 3.707e+03,    0.,  2,  2,  0)],
          dtype=[('DT', '<M8[us]'), ('Index', '<i4'), ('Price', '<f8'), ('Duration', '<m8[us]'), ('Volume', '<f8'), ('BuySize', '<f8'), ('SellSize', '<f8'), ('Count', '<i4'), ('BuyCount', '<i4'), ('SellCount', '<i4')])

In [5]:
len(T), len(C)

(2000, 201)

In [6]:
# indexed chart
fig = chartFigure(height=400,
    Sell=dict(x=Sell.DateTimeA, y=Sell.PriceA, color='red', mode='markers', size=-symlog(Sell.VolumeA)+3, opacity=0.2),
    Buy=dict(x=Buy.DateTimeA, y=Buy.PriceA, color='green', mode='markers', size=symlog(Buy.VolumeA)+3, opacity=0.2),
    Debounce=dict(mode='markers+lines', x=C.DT, y=C.Price, color='black', connectgaps=True, opacity=0.5)
)
fig

FigureWidgetResampler({
    'data': [{'marker': {'color': 'red',
                         'opacity': 0.2,
    …

In [7]:
C.Volume = C.BuySize + C.SellSize
Buy = C[C.Volume > 0]
Sell = C[C.Volume < 0]
Sell[-5:]

rec.array([('2022-05-08T00:01:36.910000', 174, 2637.915,   35999,  -902.,    0., -9.020e+02,  5,  0, 5),
           ('2022-05-08T00:01:36.955000', 176, 2637.895,       0,    -2.,    0., -2.000e+00,  1,  0, 1),
           ('2022-05-08T00:01:39.125000', 185, 2638.015,       0,    -8.,    0., -8.000e+00,  2,  0, 2),
           ('2022-05-08T00:01:44.624999', 191, 2638.135, 2117001, -1740.,  992., -2.732e+03, 26, 17, 9),
           ('2022-05-08T00:01:46.744000', 194, 2638.275,  459000,  -401., 1209., -1.610e+03,  9,  5, 4)],
          dtype=[('DT', '<M8[us]'), ('Index', '<i4'), ('Price', '<f8'), ('Duration', '<m8[us]'), ('Volume', '<f8'), ('BuySize', '<f8'), ('SellSize', '<f8'), ('Count', '<i4'), ('BuyCount', '<i4'), ('SellCount', '<i4')])

In [8]:
S = np.load('data/ETHUSDT.2022-05-08.bidask.npz')['ETHUSDT'].view(np.recarray)
S[:5]

rec.array([('2022-05-08T00:01:00.007000', 2636., 2635.99),
           ('2022-05-08T00:01:00.043000', 2636., 2635.99),
           ('2022-05-08T00:01:00.079000', 2636., 2635.99),
           ('2022-05-08T00:01:00.117000', 2636., 2635.99),
           ('2022-05-08T00:01:00.149000', 2636., 2635.99)],
          dtype={'names':['DT','A','B'], 'formats':['<M8[us]','<f4','<f4'], 'offsets':[0,8,16], 'itemsize':24})

In [9]:
K = npJoin(C.DT, S.DT)

In [10]:
# indexed chart without bidask
fig = chartFigure(height=600, rows=2,
    Sell=dict(x=Sell.Index, y=Sell.Price, color='red', mode='markers', size=-symlog(Sell.Volume)-1, opacity=0.75),
    Buy=dict(x=Buy.Index, y=Buy.Price, color='green', mode='markers', size=symlog(Buy.Volume)-1, opacity=0.75),
    Bid=dict(x=C.Index, y=S.B[K], color='green', opacity=0.4),
    Ask=dict(x=C.Index, y=S.A[K], color='red', opacity=0.4)
)
fig.add_bar(name='Buy', x=Buy.Index, y=Buy.Volume, row=2, col=1, marker_color='green')
fig.add_bar(name='Sell', x=Sell.Index, y=Sell.Volume, row=2, col=1, marker_color='red')
fig

FigureWidgetResampler({
    'data': [{'marker': {'color': 'red',
                         'opacity': 0.75,
   …